In [1]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import numpy as np
from datetime import datetime 

In [9]:
df=pd.DataFrame(data=[{'CPU': 'c1', 'QTY': 1}, {'CPU': 'c2', 'QTY': 1}])
df=df.fillna('')
#[{"CPU": "c1", "QTY": 1}, {"CPU": "c2", "QTY": 1}]
df



,CPU,QTY
0,c1,1
1,c2,1


In [2]:
start_date_query='2023-03-21'

In [3]:
# Get Last Upldate from BQ update data
dateCols=['open_datetime','close_datetime','response_datetime','resolved_datetime']

removeCols=dateCols+['open_to_close','response_to_resolved']

numbericCols=['open_to_close_hour','response_to_resolved_hour']
cateCols=['sla','product_type','brand','service_type','incident_type']

file_name="New_Incident.csv"
table_id = "pongthorn.SMartML.new_incident"


In [4]:
client = bigquery.Client()

In [5]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("sla", "STRING", mode="REQUIRED"),    
    bigquery.SchemaField("product_type", "STRING", mode="REQUIRED"),  
    bigquery.SchemaField("brand", "STRING", mode="REQUIRED"),  
    bigquery.SchemaField("service_type", "STRING", mode="REQUIRED"),  
    bigquery.SchemaField("incident_type", "STRING", mode="REQUIRED"),  
    bigquery.SchemaField("open_to_close_hour", "FLOAT", mode="REQUIRED"),
    bigquery.SchemaField("response_to_resolved_hour", "FLOAT", mode="REQUIRED"),    
    bigquery.SchemaField("imported_at", "DATETIME", mode="REQUIRED")    
    ]

    table = bigquery.Table(table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="imported_at")

    
    table = client.create_table(table)  # Make an API request.
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )


Table pongthorn.SMartML.new_incident already exists.


In [6]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )

Imported DateTime: 2023-03-27 16:05:17


In [7]:
sql_lastImport=f"SELECT max(imported_at) as last_imported from `{table_id}` "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

if str_lastImported is not None:
  print("Start date from last loading")  
  start_date_query=str_lastImported
else:
  print("Init First loading")  


print(f"Start Import on update_at of last imported date : {start_date_query}" )


Last Imported DateTime: 2023-03-27 05:27:08
Start date from last loading
Start Import on update_at of last imported date : 2023-03-27 05:27:08


In [8]:
sql=f"""
SELECT  id,
severity_id,severity_name,sla,
product_type,brand,service_type,incident_type,
open_datetime,  close_datetime, response_datetime,resolved_datetime
FROM `pongthorn.SMartDW.incident` 
WHERE imported_at>'{start_date_query}'
order by imported_at
"""
#WHERE imported_at>='{start_date_query}' and imported_at<='2023-03-24'
#WHERE imported_at>='{start_date_query}'

query_result=client.query(sql)
df_all=query_result.to_dataframe()
print(df_all.info())
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 0 non-null      object
 1   severity_id        0 non-null      object
 2   severity_name      0 non-null      object
 3   sla                0 non-null      object
 4   product_type       0 non-null      object
 5   brand              0 non-null      object
 6   service_type       0 non-null      object
 7   incident_type      0 non-null      object
 8   open_datetime      0 non-null      object
 9   close_datetime     0 non-null      object
 10  response_datetime  0 non-null      object
 11  resolved_datetime  0 non-null      object
dtypes: object(12)
memory usage: 0.0+ bytes
None


,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_datetime,close_datetime,response_datetime,resolved_datetime


In [9]:
if len(df_all)==0:
 print("No record to load")   
 exit()

No record to load


In [145]:
start_end_list=[ ['open_datetime','close_datetime'],['response_datetime','resolved_datetime']]
listDiffDateDeltaCols=[]
listDiffHourCols=[]
for item  in  start_end_list:
   diff_str=f"{item[0]}_to_{item[1]}" 
   diff_str=diff_str.replace('_datetime','')  
   listDiffDateDeltaCols.append(diff_str)
   df_all[diff_str]=df_all[item[1]]-df_all[item[0]]
    
   diff_hour=f'{diff_str}_hour'
   listDiffHourCols.append(diff_hour)
   df_all[diff_hour] = df_all[diff_str].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )


In [146]:
for col in numbericCols:
 df_all=df_all.query(f'{col}!=0')

df_all=df_all.drop_duplicates(subset=['id'],keep='first')
df_all=df_all.drop(columns=removeCols)

df_all['imported_at']=dt_imported

df_all.dropna(inplace=True)

print(df_all.info())
df_all.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 42
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         43 non-null     Int64         
 1   severity_id                43 non-null     Int64         
 2   severity_name              43 non-null     object        
 3   sla                        43 non-null     object        
 4   product_type               43 non-null     object        
 5   brand                      43 non-null     object        
 6   service_type               43 non-null     object        
 7   incident_type              43 non-null     object        
 8   open_to_close_hour         43 non-null     float64       
 9   response_to_resolved_hour  43 non-null     float64       
 10  imported_at                43 non-null     datetime64[ns]
dtypes: Int64(2), datetime64[ns](1), float64(2), object(6)
memory usage: 4.1+ 

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,imported_at
38,2472,2,Major,24x7 6Hrs Response Time,Software,Veritas,Incident,Backup Failure,141.466667,141.316667,2023-03-27 05:27:08.968019
39,2523,2,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,8.916667,8.716667,2023-03-27 05:27:08.968019
40,2522,2,Major,8x5 4Hrs Response Time,Tape Library,HPE,Incident,Controller/Node Failure,12.383333,12.150000,2023-03-27 05:27:08.968019
41,2168,2,Major,24x7 4Hrs Response Time,Server,Cisco,Incident,Fan Failure,2137.500000,2016.883333,2023-03-27 05:27:08.968019
42,2524,2,Major,8x5 4Hrs Response Time,Software,Microsoft,Incident,General Incident,16.000000,12.500000,2023-03-27 05:27:08.968019


In [147]:
print(df_all[numbericCols].describe(percentiles=[.9,.75,.50,.25,.10]))

       open_to_close_hour  response_to_resolved_hour
count           43.000000                  43.000000
mean           110.340310                  87.460853
std            351.182457                 319.776231
min              0.083333                   0.066667
10%              0.636667                   0.406667
25%              4.408333                   3.350000
50%              9.000000                   7.000000
75%             25.450000                  22.691667
90%            138.970000                 119.250000
max           2137.500000                2016.883333


In [148]:
df_all.to_csv("data/New_Incident.csv",index=False)
# dfError= df_all.query("open_to_close_hour>=524.183333")
# dfError.to_csv("Error_Incident.csv",index=False)
# dfError
# dfError['id'].tolist()

In [149]:

def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df_all, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_all), "Imported igquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex


Total  43 Imported igquery successfully
